In [1]:
import os
import pandas as pd
import numpy as np
from pathlib import Path
import geopandas as gpd
from tqdm import tqdm
from IPython.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

from calendar import monthrange
import datetime
import warnings
warnings.filterwarnings('ignore')

###### Assign random date

In [2]:
pathin = Path(r'../data/STORM/processed')
dfall = pd.read_pickle(pathin/'STORM_NA_R4.pkl')

In [3]:
dfcl = pd.read_pickle(pathin/'STORM_NA_R4_In_pnt_v7_MDAvec_12params_1000.pkl')

In [4]:
dfcl.head()

,lat_at_min_distNC,lon_at_min_distNC,min_press_at_min_distNC,rad_to_max_ws_at_min_distNC,forward_speed_at_min_distNC,landfall,trajectory_in_roi,mean_trajectory_roi,mean_curvature_roi,duration,lat_gen,lon_gen,index_cluster,weight,npoints,tc_number
0,31.2,-75.2,999.804132,37.040001,29.046809,1.0,194.036243,183.495849,0.180532,99.0,31.2,-75.2,5250,0.000102,1,57667.0
1,33.3,-71.7,947.700000,30.626958,26.617725,0.0,15.945396,28.405243,0.134638,828.0,13.4,-20.3,2454,0.000102,1,27039.0
2,30.5,-77.2,1001.600000,166.679993,9.780792,0.0,0.000000,57.804361,2.802478,48.0,29.9,-78.5,1652,0.000102,1,17999.0
3,35.6,-75.3,992.200000,171.207108,35.431070,1.0,327.994617,331.926563,0.047634,168.0,12.8,-30.5,5926,0.000204,2,65274.0
4,38.7,-73.5,995.600000,101.649548,3.700357,0.0,270.000000,168.718546,3.545342,315.0,7.5,-30.2,8556,0.000102,1,95729.0


In [4]:
## full data of selected storms
dfallsub = dfall.loc[dfall['tc_number'].isin(dfcl['tc_number']), :]

In [6]:
## count number of storms per year

In [5]:
dfall.head()

,year,month,yr_tc_number,time_step,basin_id,lat,lon,min_press,max_ws,rad_to_max_ws,cat,landfall,dist_to_land,tc_number,dist_to_NC,trajectory,forwardSpeed
0,0.0,10.0,0.0,0.0,1.0,8.9,-45.2,993.946699,20.371998,46.299999,0.0,0.0,851.946656,0,4336.133140,281.309932,18.695934
1,0.0,10.0,0.0,1.0,1.0,9.0,-45.7,992.800000,21.200000,45.905957,0.0,0.0,815.967429,0,4290.629311,278.130102,18.695934
2,0.0,10.0,0.0,2.0,1.0,9.1,-46.4,992.100000,21.700000,45.511914,0.0,0.0,766.103506,0,4230.498007,276.340192,25.916882
3,0.0,10.0,0.0,3.0,1.0,9.2,-47.3,993.000000,21.100000,45.117872,0.0,0.0,705.759651,0,4156.153101,275.710593,33.179142
4,0.0,10.0,0.0,4.0,1.0,9.3,-48.3,993.100000,20.900000,44.723829,0.0,0.0,647.548285,0,4075.371766,281.309932,36.812210


In [6]:
nstorms_per_year = dfall.groupby(['year'])['yr_tc_number'].max()

In [7]:
mean_val = nstorms_per_year.mean()
std_val = nstorms_per_year.std()

In [8]:
mean_val

9.7065

In [9]:
std_val

3.245983886457262

In [10]:
np.random.seed(42)
dfcl2 = dfcl.copy()
dfout = pd.DataFrame()
for yr in tqdm(np.arange(2020, 1800, -1)):
    ## select random number storms for year yr
    n = np.random.randint(int(mean_val - std_val), int(mean_val + std_val))
    ## select random storms
    try:
        dfcls = dfcl2.sample(n)
    except:
        ## there are less storms without date assigned than n
        dfcls = dfcl2.copy()
    
    for i in range(len(dfcls)):
        dfa = dfallsub[dfallsub['tc_number'] == dfcls['tc_number'].iloc[i]]
        ndays = monthrange(yr, int(dfa['month'].unique()))[1]
        dummy = 0
        i = 0
        while dummy == 0:
            startday = np.random.randint(1, ndays)
            hours_to_start = [0, 3, 6, 9, 12, 15, 18, 21]
            dummy2 = np.random.randint(0, len(hours_to_start))
            starthour = hours_to_start[dummy2]
            dfa.loc[:, 'date'] = pd.date_range(start = datetime.datetime(yr, int(dfa['month'].unique()), startday, starthour), 
                                               periods = len(dfa), freq = '3H')
#             dfa.loc[:, 'date'] = pd.date_range(start = datetime.datetime(yr, int(dfa['month'].unique()), startday, 0), 
#                                                periods = len(dfa), freq = '3H')
            aux = np.array([x.month for x in dfa['date']])
            l = aux[aux == dfa['month'].unique()]
            if len(l) > 3*len(dfa)/4:
                dummy = 1
#                 print(f'Done with {i} iterations')
        dfout = pd.concat([dfout, dfa])
    ## delete selected updated storms
    dfcl2 = dfcl2.drop(dfcls.index, axis = 'rows')
    
    if len(dfcl2) == 0:
        break
dfallsub['random_date'] = dfout.loc[dfallsub.index, 'date']

dfallsub.to_pickle(pathin/f'STORM_NA_R4_In_pnt_v7_MDAvec_12params_1000_map_original_ts_v2.pkl')

 54%|████████████████████████████████████████████████████                                             | 118/220 [00:02<00:02, 40.59it/s]


In [11]:
dfallsub

,year,month,yr_tc_number,time_step,basin_id,lat,lon,min_press,max_ws,rad_to_max_ws,cat,landfall,dist_to_land,tc_number,dist_to_NC,trajectory,forwardSpeed,random_date
8007,22.0,9.0,9.0,0.0,1.0,8.5,-52.4,989.319722,22.635555,64.820000,0.0,0.0,342.675367,240,3862.456146,291.801409,26.566274,1989-09-13 00:00:00
8008,22.0,9.0,9.0,1.0,1.0,8.7,-52.9,989.000000,22.900000,64.154437,0.0,0.0,341.840499,240,3812.068842,296.565051,19.772230,1989-09-13 03:00:00
8009,22.0,9.0,9.0,2.0,1.0,8.9,-53.3,986.900000,24.300000,63.488875,0.0,0.0,348.812924,240,3768.318132,296.565051,16.417868,1989-09-13 06:00:00
8010,22.0,9.0,9.0,3.0,1.0,9.1,-53.7,984.500000,25.900000,62.823312,0.0,0.0,360.138578,240,3724.677115,306.869898,16.410789,1989-09-13 09:00:00
8011,22.0,9.0,9.0,4.0,1.0,9.4,-54.1,981.000000,28.300000,62.157750,0.0,0.0,385.725476,240,3672.133302,306.869898,18.356941,1989-09-13 12:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3501006,9982.0,8.0,8.0,50.0,1.0,28.9,-15.0,984.200000,28.900000,32.277715,0.0,0.0,92.415900,109296,5785.749949,90.000000,87.777392,2014-08-21 06:00:00
3501007,9982.0,8.0,8.0,51.0,1.0,28.9,-12.2,989.200000,24.100000,33.468286,0.0,0.0,87.761055,109296,6035.038328,92.045408,91.028258,2014-08-21 09:00:00
3501008,9982.0,8.0,8.0,52.0,1.0,28.8,-9.4,996.200000,16.600000,34.658858,0.0,1.0,0.000000,109296,6287.796278,90.000000,91.146738,2014-08-21 12:00:00
3501009,9982.0,8.0,8.0,53.0,1.0,28.8,-6.3,996.200000,16.629436,35.849429,0.0,1.0,0.000000,109296,6560.995899,90.000000,100.877222,2014-08-21 15:00:00


###### Get tide information

In [12]:
pathin = Path(r'../data/STORM/processed')
df = pd.read_pickle(pathin/f'STORM_NA_R4_In_pnt_v7_MDAvec_12params_1000_map_original_ts_v3.pkl')

In [13]:
df.head()

,year,month,yr_tc_number,time_step,basin_id,lat,lon,min_press,max_ws,rad_to_max_ws,cat,landfall,dist_to_land,tc_number,dist_to_NC,trajectory,forwardSpeed,random_date
8007,22.0,9.0,9.0,0.0,1.0,8.5,-52.4,989.319722,22.635555,64.820000,0.0,0.0,342.675367,240,3862.456146,291.801409,26.566274,1989-09-13 00:00:00
8008,22.0,9.0,9.0,1.0,1.0,8.7,-52.9,989.000000,22.900000,64.154437,0.0,0.0,341.840499,240,3812.068842,296.565051,19.772230,1989-09-13 03:00:00
8009,22.0,9.0,9.0,2.0,1.0,8.9,-53.3,986.900000,24.300000,63.488875,0.0,0.0,348.812924,240,3768.318132,296.565051,16.417868,1989-09-13 06:00:00
8010,22.0,9.0,9.0,3.0,1.0,9.1,-53.7,984.500000,25.900000,62.823312,0.0,0.0,360.138578,240,3724.677115,306.869898,16.410789,1989-09-13 09:00:00
8011,22.0,9.0,9.0,4.0,1.0,9.4,-54.1,981.000000,28.300000,62.157750,0.0,0.0,385.725476,240,3672.133302,306.869898,18.356941,1989-09-13 12:00:00


In [14]:
for itc, tc in tqdm(enumerate(df['tc_number'].unique())):
    dfs = df[df['tc_number'] == tc]
    dur = (dfs['random_date'].iloc[-1] - dfs['random_date'].iloc[0]).total_seconds()/86400 + 3
    hh = dfs['random_date'].iloc[0].hour
    dd = dfs['random_date'].iloc[0].day
    mm = dfs['random_date'].iloc[0].month
    yy = dfs['random_date'].iloc[0].year
    with open(r'TideFac-Code-Executable/dates.txt', 'w') as fout:
        fout.write(f'{dur:0.3f}\n')
        fout.write(f'{hh:02d},{dd},{mm},{yy}')
    os.system(r'TideFac-Code-Executable/a.out < TideFac-Code-Executable/dates.txt > TideFac-Code-Executable/screen.txt')
    
    aux = pd.read_csv('tide_fac.out', skiprows=9, header = None, delim_whitespace = True,
                           names = [f'node_factor_{int(tc)}', f'eq_arg_{int(tc)}'])
    if itc == 0:
        dfout = aux.copy()
    else:
        dfout = pd.concat([dfout, aux], axis = 1)
try:
    os.remove('tide_fac.out')
except:
    pass
dfout.to_pickle(pathin/f'STORM_NA_R4_In_pnt_v7_MDAvec_12params_1000_map_original_ts_v3_tide_data.pkl')

1000it [00:16, 62.02it/s]
